### Adjust Word Level

In [34]:
#!pip install xlsxwriter

In [35]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from pathlib import Path
import shutil

In [36]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


#### While Loop

In [37]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word

# word frequency
word_frequency = True  # True, False

In [38]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/3-Adjust Word Level/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)  # create path

##### Dependency DataFrames

In [39]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
988212,karneleme,5
988213,karnaya,5
988214,dörtlümüzün,5
988215,karnavalınız,5


In [40]:
#df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")
df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
df_twogram_sent

,twogram,frequency
0,teşekkür ederim,244149
1,öyle mi,209900
2,ne oldu,195799
3,aman tanrım,189521
4,özür dilerim,153784
...,...,...
1036515,güzeldi tommy,3
1036516,durumu tuhaflaştırma,3
1036517,güzeldi canım,3
1036518,güzeldi daniel,3


In [41]:
#lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
#lang_pair_list

In [42]:
#df_pair_ety = pd.read_excel(f"{lang_pair_list[0]}")  # need only dict_entry_main column
df_pair_ety = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.lower().capitalize()}/{lang_folder.capitalize()}_{lang_pair.lower().capitalize()}_Shared_Vocabulary.xlsx")
df_pair_ety

,dict_entry_main,english_word
0,abaküs,abacus
1,abluka,blockade
2,absorbe,absorb
3,absürt,absurd
4,açelya,azalea
...,...,...
1773,zebra,zebra
1774,zikzak,zigzag
1775,zombi,zombie
1776,zooloji,zoology


In [43]:
# Option
if prefix_suffix:
    df_prefix_suffix_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.lower().capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
    df_prefix_suffix_select = df_prefix_suffix_select.loc[:,["search_word","word"]]
    df_prefix_suffix_select.rename(columns={"search_word":"dict_entry_main"}, inplace=True)
    df_pair_merge = pd.merge(df_pair_ety,df_prefix_suffix_select, how="inner", on="dict_entry_main")
    df_pair_merge.drop_duplicates(inplace=True)
    df_pair_merge.reset_index(drop=True, inplace=True)
    df_pair_merge = df_pair_merge.loc[:,["word",f"{lang_pair.lower()}_word"]]
    df_pair_merge.rename(columns={"word":"dict_entry_main"}, inplace=True)
    df_pair_all = df_pair_merge
else:
    df_pair_all = df_pair_ety

df_pair_all

,dict_entry_main,english_word
0,abaküs,abacus
1,abluka,blockade
2,absorbe,absorb
3,absürt,absurd
4,açelya,azalea
...,...,...
1773,zebra,zebra
1774,zikzak,zigzag
1775,zombi,zombie
1776,zooloji,zoology


In [44]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [45]:
words = df_pair_all["dict_entry_main"].values.tolist()
disable_set = set(disable_list)
words_set = set(words)

In [46]:
df_pair = pd.DataFrame(list(words_set.difference(disable_set)), columns=["dict_entry_main"])
df_pair

,dict_entry_main
0,hokus pokus
1,totaliter
2,dolar
3,repo
4,dinamit
...,...
1771,printer
1772,mobil
1773,bisküvi
1774,hemoroid


##### Dependency Functions

In [47]:
def remove_repetition(word_group):
    '''
    remove_repetition(word_group): detect word repetion in word group 
    '''
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [48]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [49]:
def word_count_bool(df, word_thresh_num, column_list): # df is a dataframe, word_thresh_num is an integer, column_list is list value
    '''
    word_count_bool(df, word_thresh_num, column_list):\n
    df is a dataframe, word_thresh_num is an integer, column_list is df column names\n
    word_count_bool(df, 7, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"count"}, inplace=True)
    
    return  df_word_count["count"][df_word_count.loc[:,"count"] > word_thresh_num].any()  

#### Condition 1

In [50]:
def condition1(word_start_num = 0, word_limit_num = 28, thresh_num = 2, total_part_num = 4):
    '''
    default parameter:\n    
    condition1(word_start_num = 0, word_limit_num = 28, thresh_num = 2, total_part_num = 4) \n
    word_end 28/4 = 7 word group
    '''

    # while loop code block word and twogram pair
    word_thresh_num = thresh_num  # want how many word sample 
    word_start = word_start_num  # 0
    word_end = int((word_limit_num-word_start_num)/total_part_num)  
    step_num = word_end  
    word_limit = word_limit_num   
    part_num = 1  # first output file extention
    
    #twogram_num = word_thresh_num * step_num  # word_thresh_num*step_num minimum: for each word takes two twogram
    twogram_pair_num = word_thresh_num * step_num  # word_thresh_num*step_num minimum: for each word takes two twogram pair
    
    while word_end <= word_limit:
        df_word = df_word_all.iloc[word_start:word_end,]  # must be include word and frequency column
        df_word.reset_index(drop=True, inplace=True)
    
        # language pair twogram
        word_pair_list = df_pair["dict_entry_main"].to_list()  # *****
        word_list = df_word["word"].to_list()  # *****    
        ngram_list = []
        for i in word_pair_list:
            for j in word_tokenize(i):
                for k in word_list:
                    twogram_1_2 = f"{j} {k}"
                    ngram_list.append(twogram_1_2)
                    twogram_2_1 = f"{k} {j}"
                    ngram_list.append(twogram_2_1)
        df_pair_ngram = pd.DataFrame(ngram_list, columns=["twogram"])
        #df_pair_ngram.rename(columns={0:"twogram"}, inplace=True)  # ******
        df_pair_ngram.iloc[:,0] = df_pair_ngram.iloc[:,0].apply(lambda x: remove_repetition(x))
        df_pair_ngram.drop_duplicates(inplace=True)
        df_pair_ngram.reset_index(drop=True, inplace=True)
        df_lang_pair_twogram = pd.merge(df_twogram_sent, df_pair_ngram, how="inner", on="twogram")
        df_lang_pair_twogram.rename(columns={"twogram":f"twogram_pair_{lang_pair.lower()}"}, inplace=True)
        df_lang_pair_twogram.drop_duplicates(inplace=True)
        #df_lang_pair_twogram = df_lang_pair_twogram.head(100)
    
        # output
        df_output_result = pd.concat([df_word, df_lang_pair_twogram], axis=1)
    
        df_lesson_result = pd.DataFrame(columns=["word","freq_word",f"twogram_pair_{lang_pair.lower()}",f"freq_twogram_pair_{lang_pair.lower()}"])
        a = 0
        #for i in range(0,110):
        for i in range(len(df_output_result)):  # *****
            # Insert words and their count 
            try:
                word = df_output_result.iloc[i,0]  # word 
                freq_word = df_output_result.iloc[i,1]  # word freq
                df_lesson_result.loc[i,"word"] = word
                df_lesson_result.loc[i,"freq_word"] = freq_word
            except:
                pass
            
            # Insert twogram pair
            try:
                var2 = df_output_result.loc[a,f"twogram_pair_{lang_pair.lower()}"]
                freq_var2 = df_output_result.iloc[a,3]  # twogram_pair frequency
                if (len(df_lesson_result[f"twogram_pair_{lang_pair.lower()}"]) < twogram_pair_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
                    df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                    df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                    try:
                        while word_count_bool(df_lesson_result, word_thresh_num, [f"twogram_pair_{lang_pair.lower()}"]): # word count result                
                            a += 1
                            var2 = df_output_result.loc[a,f"twogram_pair_{lang_pair.lower()}"]
                            freq_var2 = df_output_result.iloc[a,3]  # twogram_pair frequency
                            df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                            df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                        else:
                            pass
                    except:
                        df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = np.nan
                        df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = np.nan
                else:
                    pass
            except:
                pass
            a += 1
    
        df_lesson_word_count = word_count_result(df_lesson_result, [f"twogram_pair_{lang_pair.lower()}"])
        df_lesson_result = pd.merge(df_lesson_result, df_lesson_word_count, how="left", on="word")
        df_lesson_result = df_lesson_result.drop_duplicates()
        df_lesson_result.to_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result{part_num}.xlsx", index=False)
    
        word_start += step_num
        word_end += step_num
        part_num += 1 

#### Condition 2

In [51]:
# --- Parameter Part ---

# condition 1
condition1_word_start = 0
condition1_word_limit = 28
condition1_word_thresh_num = 2
condition1_total_part_num = 4  # 28/4 = 7 word group

# condition 2
condition1_dependency = True  # True, False
word_thresh_num = 20
twogram_thresh_minus = 0  # for optional twogram thresh number 
twogram_pair_thresh_minus = 0  # for optinal twogram pair thresh number.

word_start = 0  # 0
word_end = condition1_word_limit  # condition1_word_limit must be equal word_end (condition2 parameter)
step_num = word_end  # 10
word_limit = word_end  # 200
part_num = 1
    
if condition1_dependency:        
    # Read previous part result
    condition1(word_start_num = condition1_word_start, word_limit_num = condition1_word_limit, thresh_num = condition1_word_thresh_num, total_part_num = condition1_total_part_num)  # word_end 28/4 = 7 word group. Condition 1 parameters 
    df_part_all = pd.DataFrame()
    part_result_file = glob.glob(f"{lang_folder}_{lang_pair}_*_Word_Step_*_Result*.xlsx")
    for i in part_result_file:
        df_var = pd.read_excel(f"{i}")
        df_part_all = pd.concat([df_part_all,df_var], axis=0)
    df_part_twogram_pair = df_part_all.loc[:,[f"twogram_pair_{lang_pair.lower()}"]]
    df_part_twogram_pair.reset_index(drop=True, inplace=True)
    set_part_twogram_pair = set(df_part_twogram_pair[f"twogram_pair_{lang_pair.lower()}"])
else:
    set_part_twogram_pair = set([])  # option for skip Condition 1
# --- Parameter End ---

# while loop code block

twogram_num = word_thresh_num * step_num   # word_thresh_num*step_num minimum: for each word takes two twogram
twogram_pair_num = word_thresh_num * step_num  # word_thresh_num*step_num minimum: for each word takes two twogram pair

while word_end <= word_limit:
    df_word = df_word_all.iloc[word_start:word_end,]
    df_word.reset_index(drop=True, inplace=True)

    # language pair twogram
    word_pair_list = df_pair["dict_entry_main"].to_list()  # *****
    word_list = df_word["word"].to_list()  # ***** 
    ngram_list = []
    for i in word_pair_list:
        for j in word_tokenize(i):
            for k in word_list:
                twogram_1_2 = f"{j} {k}"
                ngram_list.append(twogram_1_2)
                twogram_2_1 = f"{k} {j}"
                ngram_list.append(twogram_2_1)
    df_pair_ngram = pd.DataFrame(ngram_list)
    df_pair_ngram = pd.DataFrame(ngram_list, columns=["twogram"])
    #df_pair_ngram.rename(columns={0:"twogram"}, inplace=True)  # *****
    df_pair_ngram.iloc[:,0] = df_pair_ngram.iloc[:,0].apply(lambda x: remove_repetition(x))
    df_pair_ngram.drop_duplicates(inplace=True)
    df_pair_ngram.reset_index(drop=True, inplace=True)
    df_lang_pair_twogram = pd.merge(df_twogram_sent, df_pair_ngram, how="inner", on="twogram")
    df_lang_pair_twogram.rename(columns={"twogram":f"twogram_pair_{lang_pair.lower()}"}, inplace=True)
    df_lang_pair_twogram.drop_duplicates(inplace=True)
    #df_lang_pair_twogram = df_lang_pair_twogram.head(100)
    set_lang_pair_twogram = set(df_lang_pair_twogram[f"twogram_pair_{lang_pair.lower()}"])
    df_set_result = pd.DataFrame(set_lang_pair_twogram.difference(set_part_twogram_pair))
    df_set_result.rename(columns={0:f"twogram_pair_{lang_pair.lower()}"}, inplace=True)
    df_set_pair_twogram = pd.merge(df_lang_pair_twogram, df_set_result, how="inner", on=f"twogram_pair_{lang_pair.lower()}")

    # twogram
    word_list = df_word["word"].values.tolist()
    data_kind = "twogram"
    twogram_list  = df_twogram_sent.iloc[:,0].values.tolist()
    
    resultlist2 = []

    manager = multiprocessing.Manager()
    resultlist2 = manager.list()
    
    def word_in_wordgroup2(list_var2):
        mergelist = []
        try:
            word = list_var2.split()
        except:
            pass
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist2.append(list_var2)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup2, twogram_list) # string_word liste 

    result_list2 = list(resultlist2)
    df_result2 = pd.DataFrame(result_list2)
    df_result2 = pd.DataFrame(result_list2, columns=[f"{data_kind}"])  # *****
    #df_result2 = df_result2.rename(columns = {0: f"{data_kind}"})  # *****
    df_result2.iloc[:,0] = df_result2.iloc[:,0].apply(lambda x: remove_repetition(x)) # **
    df_merge2 = pd.merge(df_result2, df_twogram_sent, how="inner", on=f"{data_kind}")
    df_merge_result2 = df_merge2.sort_values(by="frequency", ascending=False)
    df_merge_result2.drop_duplicates(inplace=True)
    df_merge_result2.reset_index(drop=True, inplace=True)
    df_twogram_result = df_merge_result2
    #df_twogram_result = df_twogram_result.head(100)

    # output
    df_output_result = pd.concat([df_word, df_twogram_result, df_set_pair_twogram], axis=1)

    df_lesson_result = pd.DataFrame(columns=["word","freq_word","twogram","freq_twogram",f"twogram_pair_{lang_pair.lower()}",f"freq_twogram_pair_{lang_pair.lower()}"])
    a = 0
    b = 0

    for i in range(len(df_output_result)):  # *****
        # Insert words and their count 
        try:
            word = df_output_result.iloc[i,0]  # word
            freq_word = df_output_result.iloc[i,1]  # word freq
            df_lesson_result.loc[i,"word"] = word
            df_lesson_result.loc[i,"freq_word"] = freq_word
        except:
            pass
         
        # Insert n grams
        try:
            var1 = df_output_result.iloc[a,2]
            freq_var1 = df_output_result.iloc[a,3]
            if (len(df_lesson_result["twogram"]) < twogram_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
                df_lesson_result.loc[i,"twogram"] = var1
                df_lesson_result.loc[i,"freq_twogram"] = freq_var1
                try:
                    while word_count_bool(df_lesson_result, (word_thresh_num - twogram_thresh_minus), ["twogram"]): # word count result                
                        a += 1
                        var1 = df_output_result.iloc[a,2]
                        freq_var1 = df_output_result.iloc[a,3]
                        df_lesson_result.loc[i,"twogram"] = var1
                        df_lesson_result.loc[i,"freq_twogram"] = freq_var1
                    else:
                        pass
                except:
                    df_lesson_result.loc[i,"twogram"] = np.nan
                    df_lesson_result.loc[i,"freq_twogram"] = np.nan
            else:
                pass
        except:
            pass
        a += 1

        try:
            var2 = df_output_result.iloc[b,4]
            freq_var2 = df_output_result.iloc[b,5]
            if (len(df_lesson_result[f"twogram_pair_{lang_pair.lower()}"]) < twogram_pair_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
                df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                try:
                    while word_count_bool(df_lesson_result, (word_thresh_num - twogram_pair_thresh_minus), [f"twogram_pair_{lang_pair.lower()}"]): # word count result                
                        b += 1
                        var2 = df_output_result.iloc[b,4]
                        freq_var2 = df_output_result.iloc[b,5]
                        df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                        df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                    else:
                        pass
                except:
                    df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = np.nan
                    df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = np.nan
            else:
                pass
        except:
            pass
        b += 1

    df_lesson_word_count = word_count_result(df_lesson_result, ["twogram",f"twogram_pair_{lang_pair.lower()}"])
    df_lesson_result = pd.merge(df_lesson_result, df_lesson_word_count, how="left", on="word")
    df_lesson_result = df_lesson_result.drop_duplicates()
    df_lesson_result.to_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result{part_num}.xlsx", index=False)

    word_start += step_num
    word_end += step_num
    part_num += 1 

#### Check Result

In [52]:
df_lesson_result

,word,freq_word,twogram,freq_twogram,twogram_pair_english,freq_twogram_pair_english,word_count
0,bir,18835735.0,ne var,62532.0,çok komik,13097,40.0
1,bu,11062659.0,ben de,59972.0,polis mi,2526,40.0
2,ne,8025880.0,değil mi,58386.0,komik değil,1785,40.0
3,ve,7766036.0,ben mi,33652.0,problem değil,1678,40.0
4,için,5484109.0,ne için,31857.0,evet patron,1476,25.0
...,...,...,...,...,...,...,...
470,NaN,NaN,NaN,NaN,politika da,3,NaN
471,NaN,NaN,NaN,NaN,sen maestro,3,NaN
472,NaN,NaN,NaN,NaN,sen doktora,3,NaN
473,NaN,NaN,NaN,NaN,sauna da,3,NaN


In [53]:
#word_count_result(df_lesson_result, ["twogram"])

In [54]:
#word_count_result(df_lesson_result, [f"twogram_pair_{lang_pair.lower()}"])

#### Read Output File And Multi Write

In [55]:
try:
    writer = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Result_With_Frequency.xlsx", engine='xlsxwriter')
    for i in range(1, (condition1_total_part_num+1)):        
        if condition1_dependency:
            df_part_var = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{int(condition1_word_limit/condition1_total_part_num)}_Result{i}.xlsx")
            df_part_var.to_excel(writer, sheet_name=f'Word_Part1{i}', index=False)            
        else:
            pass
except:
    pass

df_part_var2 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result1.xlsx")
df_part_var2.to_excel(writer, sheet_name=f'Word_Part21', index=False)        
writer.save()

#### Output File Word Order

In [56]:
def word_in_twogram(df, list_column, target_column):

    '''word_in_twogram(df, "word", "twogram"):
       df is dataframe, list_column and target_column are 
       dataframe column string name. list_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{list_column}"].dropna():
        try:
            word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_twogram.insert(0,"word",i)
        df_word_result = pd.concat([df_word_result,word_in_twogram], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [57]:
try:
    if word_frequency:
        writer2 = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Join_Result_With_Frequency.xlsx", engine='xlsxwriter')
    else:
        writer2 = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Join_Result_Without_Frequency.xlsx", engine='xlsxwriter')
    for i in range(1, (condition1_total_part_num+1)):        
        if condition1_dependency:
            df_part_var = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{int(condition1_word_limit/condition1_total_part_num)}_Result{i}.xlsx")
            df_part_var_order = word_in_twogram(df_part_var, "word", f"twogram_pair_{lang_pair.lower()}")
            df_part_var_order_join = df_part_var_order.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()
            df_part_var_order_join.to_excel(writer2, sheet_name=f'Word_Part1{i}', index=False)          
        else:
            pass
except:
    pass

df_part_var2 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result1.xlsx")
df_word_order_21 = word_in_twogram(df_part_var2, "word", f"twogram")
df_word_order_212 = word_in_twogram(df_part_var2, "word", f"twogram_pair_{lang_pair.lower()}")
df_word_order_join_211 = df_word_order_21.groupby(["word"])["twogram"].apply(", ".join).reset_index()
df_word_order_join_212 = df_word_order_212.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()
df_word_order_join_21 = pd.merge(df_word_order_join_211, df_word_order_join_212, how="outer", on="word")
df_word_order_join_21.to_excel(writer2, sheet_name='Word_Part21', index=False)        
writer2.save()

In [58]:
df_word_order_join_21

,word,twogram,twogram_pair_english
0,ama,"evet ama, ama ben, ama ne, ama bu, ama sen, ha...","pardon ama, ama doktor, ama şef, ama patron, k..."
1,bana,"bana mı, bana ne, bana da, bana değil, bu bana...","telefon bana, vay bana, bana kahve, bana bira,..."
2,ben,"ben de, ben mi, hayır ben, ama ben, ben değil,...","pardon ben, ben ray, ben kaptan, ben doktor, k..."
3,beni,"beni mi, beni de, beni değil, ve beni, sen ben...",NaN
4,bir,"bir daha, bir ne, bu bir, bir şey, bir de, o b...","bir numara, bir polis, bir milyon, bir bebek, ..."
5,bu,"bu ne, bu o, bu kadar, ne bu, ama bu, bu da, b...","bu komik, bu normal, plan bu, bu süper, bu müz..."
6,bunu,"bunu da, ve bunu, ben bunu, bunu değil, bunu i...",NaN
7,da,"bu da, bana da, o da, bunu da, onu da, daha da...","pardon da, pantolon da, telefon da, kanal da, ..."
8,daha,"daha değil, bir daha, daha çok, daha var, daha...","daha bebek, daha dramatik, daha romantik, daha..."
9,de,"ben de, sen de, seni de, bir de, evet de, beni...","bebek de, komik de, alo de, patates de, polis ..."


In [59]:
df_word_order_join_21 = pd.merge(df_word_order_join_21, df_word_all, how="left", on="word")
df_word_order_join_21.drop_duplicates(inplace=True)
df_word_order_join_21 = df_word_order_join_21.iloc[:,[0,3,1,2]]
df_word_order_join_21.sort_values(by="frequency", ascending=False, inplace=True)
df_word_order_join_21

,word,frequency,twogram,twogram_pair_english
4,bir,18835735,"bir daha, bir ne, bu bir, bir şey, bir de, o b...","bir numara, bir polis, bir milyon, bir bebek, ..."
5,bu,11062659,"bu ne, bu o, bu kadar, ne bu, ama bu, bu da, b...","bu komik, bu normal, plan bu, bu süper, bu müz..."
18,ne,8025880,"ne var, ne için, bu ne, ne kadar, ne gibi, o n...","ne komik, plan ne, problem ne, şifre ne, ne şa..."
24,ve,7766036,"ve sen, ve ben, ve bu, ve ne, ve o, ve evet, v...","ve motor, ve gol, ve bebek, ve müzik, ve komik..."
14,için,5484109,"ne için, şey için, bunu için, için mi, yok için","şans için, bebek için, parti için, okul için, ..."
16,mi,5362714,"değil mi, ben mi, sen mi, evet mi, beni mi, ne...","polis mi, bebek mi, komik mi, parti mi, kahve ..."
19,o,5013838,"bu o, o ne, o kadar, evet o, ne o, o değil, am...","patron o, o polis, o doktor, kaptan o, o norma..."
2,ben,4908913,"ben de, ben mi, hayır ben, ama ben, ben değil,...","pardon ben, ben ray, ben kaptan, ben doktor, k..."
9,de,4880315,"ben de, sen de, seni de, bir de, evet de, beni...","bebek de, komik de, alo de, patates de, polis ..."
26,çok,4852169,"bu çok, çok değil, daha çok, evet çok, sen çok...","çok komik, çok şeker, çok romantik, çok entere..."


In [60]:
df_word_order_join_21.to_excel("Turkish_English_28_Word_Join_Result_With_Frequency.xlsx", index=False)

#### Copy Move And Delete

In [22]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_limit}_Word_*.xlsx")
output_file

['Turkish_English_28_Word_Step_28_Result1.xlsx',
 'Turkish_English_28_Word_Result_With_Frequency.xlsx',
 'Turkish_English_28_Word_Join_Result_Without_Frequency.xlsx']

In [23]:
for k in output_file:
    source = k # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/3-Adjust Word Level/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [24]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass